In [1]:
import collections
import pandas as pd
import datetime
import numpy as np
import json

In [2]:
data_fr = pd.read_csv('../data/df_fr.csv', sep=',')
corres_ICD10 = pd.read_csv('../data/corres_ICD10.csv', sep=';',usecols=[0,1,2,4], header=None)

In [9]:
corres_ICD10.groupby([0,1,2,4]).sum()

Empty DataFrame
Columns: []
Index: [(A-R_V-Y, .root, ..root, Toutes causes de mortalit� (A00-Y89) sauf S00-T98), (A-R_V-Y, A_U, ..root, Causes interne de morbidit� et mortalit� ), (A-R_V-Y, A_U, A_B, Autres certaines maladies infectieuses et parasitaires (partie restante de A00-B99)), (A-R_V-Y, A_U, A_B, Certaines maladies infectieuses et parasitaires (A00-B99)), (A-R_V-Y, A_U, A_B, H�patite virale et s�quelles d'h�patite virale), (A-R_V-Y, A_U, A_B, Maladies dues au virus de l'immunod�ficience humaine [VIH]), (A-R_V-Y, A_U, A_B, Tuberculose), (A-R_V-Y, A_U, C, Autre tumeurs malignes des tissus lympho�de, h�matopo��tique et apparent�s), (A-R_V-Y, A_U, C, Autres tumeurs malignes (partie restante de C00-C97)), (A-R_V-Y, A_U, C, Leuc�mie), (A-R_V-Y, A_U, C, Maladie de Hodgkin et lymphomes), (A-R_V-Y, A_U, C, M�lanome malin de la peau), (A-R_V-Y, A_U, C, Tumeur maligne d'autres parties de l'ut�rus), (A-R_V-Y, A_U, C, Tumeur maligne de l'estomac), (A-R_V-Y, A_U, C, Tumeur maligne de l'oesophage), (A-R_V-Y, A_U, C, Tumeur maligne de l'ovaire), (A-R_V-Y, A_U, C, Tumeur maligne de la l�vre, de la cavit� buccale et du pharynx), (A-R_V-Y, A_U, C, Tumeur maligne de la prostate), (A-R_V-Y, A_U, C, Tumeur maligne de la thyro�de), (A-R_V-Y, A_U, C, Tumeur maligne de la trach�e et des bronches et du poumon), (A-R_V-Y, A_U, C, Tumeur maligne de la vessie), (A-R_V-Y, A_U, C, Tumeur maligne du cerveau et du syst�me nerveux central), (A-R_V-Y, A_U, C, Tumeur maligne du col de l'ut�rus), (A-R_V-Y, A_U, C, Tumeur maligne du c�lon, de la jonction recto-sigmo�dienne, du rectum, de l'anus et du canal anal), (A-R_V-Y, A_U, C, Tumeur maligne du foie et des voies biliaires intrah�patiques), (A-R_V-Y, A_U, C, Tumeur maligne du larynx), (A-R_V-Y, A_U, C, Tumeur maligne du pancr�as), (A-R_V-Y, A_U, C, Tumeur maligne du rein, � l'exception du bassinet), (A-R_V-Y, A_U, C, Tumeur maligne du sein), (A-R_V-Y, A_U, C, Tumeurs), (A-R_V-Y, A_U, C, Tumeurs malignes (C00-C97)), (A-R_V-Y, A_U, D, Maladies du sang et des organes h�matopo��tiques et certains troubles du syst�me immunitaire), (A-R_V-Y, A_U, D, Tumeurs non malignes (b�nignes et incertaines)), (A-R_V-Y, A_U, E, Autres maladies endocriniennes, nutritionnelles et m�taboliques (partie restante de E00-E90)), (A-R_V-Y, A_U, E, Diab�te sucr�), (A-R_V-Y, A_U, E, Maladies endocriniennes, nutritionnelles et m�taboliques (E00-E90)), (A-R_V-Y, A_U, F, Autres troubles mentaux et du comportement (partie restante de F00-F99)), (A-R_V-Y, A_U, F, D�mence), (A-R_V-Y, A_U, F, Troubles mentaux et du comportement (F00-F99)), (A-R_V-Y, A_U, F, Troubles mentaux et du comportement li�s � l'utilisation d'alcool), (A-R_V-Y, A_U, G, Autres maladies du syst�me nerveux et des organes des sens (partie restante de G00-H95)), (A-R_V-Y, A_U, G, Maladie d'Alzheimer), (A-R_V-Y, A_U, G, Maladie de Parkinson), (A-R_V-Y, A_U, G, Maladies du syst�me nerveux et des organes des sens (G00-H95)), (A-R_V-Y, A_U, I, Autres cardiopathies isch�miques), (A-R_V-Y, A_U, I, Autres maladies cardiovasculaires), (A-R_V-Y, A_U, I, Autres maladies de l'appareil circulatoire (partie restante de I00-I99)), (A-R_V-Y, A_U, I, Cardiopathies isch�miques), (A-R_V-Y, A_U, I, Infarctus aigu du myocarde et infarctus du myocarde � r�p�tition), (A-R_V-Y, A_U, I, Maladies c�r�brovasculaires), (A-R_V-Y, A_U, I, Maladies de l'appareil circulatoire (I00-I99)), (A-R_V-Y, A_U, J, Asthme et �tat de mal asthmatique), (A-R_V-Y, A_U, J, Autres maladie de l�appareil respiratoire (partie restante de J00-J99)), (A-R_V-Y, A_U, J, Autres maladies des voies respiratoires inf�rieures), (A-R_V-Y, A_U, J, Grippe (grippe porcine incluse)), (A-R_V-Y, A_U, J, Maladies chroniques des voies respiratoires inf�rieures), (A-R_V-Y, A_U, J, Maladies de l'appareil respiratoire (J00-J99)), (A-R_V-Y, A_U, J, Pneumonie), (A-R_V-Y, A_U, K, Autres maladies de l'appareil digestif (partie restante de K00-K93)), (A-R_V-Y, A_U, K, Maladie chronique du foie), (A-R_V-Y, A_U, K, Maladies de l'appareil digestif (